In [1]:
import os
import random
import math
import networkx as nx
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle

######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./SiouxFalls/SiouxFalls_net.tntp"
trips_file   = "./SiouxFalls/SiouxFalls_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. Flow 데이터 로드 및 NetworkX 변환
######################################

flow_file_path = "./SiouxFalls/SiouxFalls_flow.tntp"
flow_df = pd.read_csv(flow_file_path, sep='\s+')
flow_df.columns = ["Init node", "Term node", "Volume", "Cost"]

flow_cost = {}
for _, row in flow_df.iterrows():
    u = int(row["Init node"])
    v = int(row["Term node"])
    flow_cost[(u, v)] = float(row["Cost"])

G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        travel_time = flow_cost.get((u, v), free_time)
        G_nx.add_edge(u, v, cost=travel_time)
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['cost'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=50):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='cost')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 1000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_SiouxFalls_congestion.pkl", "wb") as f:
    pickle.dump(training_dataset, f)


Loaded 24 nodes, 76 directed links, and 528 OD pairs (o != d).


In [2]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./SiouxFalls/SiouxFalls_net.tntp"
trips_file   = "./SiouxFalls/SiouxFalls_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. NetworkX 그래프로 변환 및 경로 계산 함수 정의
######################################
G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        G_nx.add_edge(u, v, free_flow=free_time)
        
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['free_flow'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=50):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='free_flow')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 1000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_SiouxFalls_freeflow.pkl", "wb") as f:
    pickle.dump(training_dataset, f)


Loaded 24 nodes, 76 directed links, and 528 OD pairs (o != d).


In [3]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Anaheim/Anaheim_net.tntp"
trips_file   = "./Anaheim/Anaheim_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. Flow 데이터 로드 및 NetworkX 변환
######################################

flow_file_path = "./Anaheim/Anaheim_flow.tntp"
flow_df = pd.read_csv(flow_file_path, sep='\s+')
flow_df.columns = ["Init node", "Term node", "Volume", "Cost"]

flow_cost = {}
for _, row in flow_df.iterrows():
    u = int(row["Init node"])
    v = int(row["Term node"])
    flow_cost[(u, v)] = float(row["Cost"])

G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        travel_time = flow_cost.get((u, v), free_time)
        G_nx.add_edge(u, v, cost=travel_time)
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['cost'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=200):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='cost')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Anaheim_congestion.pkl", "wb") as f:
    pickle.dump(training_dataset, f)

Loaded 416 nodes, 914 directed links, and 1406 OD pairs (o != d).


In [4]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Anaheim/Anaheim_net.tntp"
trips_file   = "./Anaheim/Anaheim_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. NetworkX 그래프로 변환 및 경로 계산 함수 정의
######################################
G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        G_nx.add_edge(u, v, free_flow=free_time)
        
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['free_flow'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=50):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='free_flow')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Anaheim_freeflow.pkl", "wb") as f:
    pickle.dump(training_dataset, f)


Loaded 416 nodes, 914 directed links, and 1406 OD pairs (o != d).


In [5]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Barcelona/Barcelona_net.tntp"
trips_file   = "./Barcelona/Barcelona_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. Flow 데이터 로드 및 NetworkX 변환
######################################

flow_file_path = "./Barcelona/Barcelona_flow.tntp"
flow_df = pd.read_csv(flow_file_path, sep='\s+')
flow_df.columns = ["Init node", "Term node", "Volume", "Cost"]

flow_cost = {}
for _, row in flow_df.iterrows():
    u = int(row["Init node"])
    v = int(row["Term node"])
    flow_cost[(u, v)] = float(row["Cost"])

G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        travel_time = flow_cost.get((u, v), free_time)
        G_nx.add_edge(u, v, cost=travel_time)
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['cost'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=200):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='cost')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Barcelona_congestion.pkl", "wb") as f:
    pickle.dump(training_dataset, f)

Loaded 930 nodes, 2522 directed links, and 7922 OD pairs (o != d).


In [6]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Barcelona/Barcelona_net.tntp"
trips_file   = "./Barcelona/Barcelona_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. NetworkX 그래프로 변환 및 경로 계산 함수 정의
######################################
G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        G_nx.add_edge(u, v, free_flow=free_time)
        
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['free_flow'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=50):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='free_flow')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Barcelona_freeflow.pkl", "wb") as f:
    pickle.dump(training_dataset, f)


Loaded 930 nodes, 2522 directed links, and 7922 OD pairs (o != d).


In [8]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Chicago-Sketch/ChicagoSketch_net.tntp"
trips_file   = "./Chicago-Sketch/ChicagoSketch_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. Flow 데이터 로드 및 NetworkX 변환
######################################

flow_file_path = "./Chicago-Sketch/ChicagoSketch_flow.tntp"
flow_df = pd.read_csv(flow_file_path, sep='\s+')
flow_df.columns = ["Init node", "Term node", "Volume", "Cost"]

flow_cost = {}
for _, row in flow_df.iterrows():
    u = int(row["Init node"])
    v = int(row["Term node"])
    flow_cost[(u, v)] = float(row["Cost"])

G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        travel_time = flow_cost.get((u, v), free_time)
        G_nx.add_edge(u, v, cost=travel_time)
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['cost'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=200):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='cost')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Chicago_congestion.pkl", "wb") as f:
    pickle.dump(training_dataset, f)

Loaded 933 nodes, 2950 directed links, and 93135 OD pairs (o != d).


In [9]:
######################################
# 1. 네트워크 및 OD 데이터 로드
######################################

network_file = "./Chicago-Sketch/ChicagoSketch_net.tntp"
trips_file   = "./Chicago-Sketch/ChicagoSketch_trips.tntp"

graph = {}            # node -> list of (neighbor, travel_time)
free_flow_times = {}  # (u,v) -> free flow travel time

with open(network_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            break
        if line.endswith(";"):
            line = line[:-1]
        parts = line.split()
        if len(parts) >= 5:
            u = int(parts[0])
            v = int(parts[1])
            free_time = float(parts[4])
            free_flow_times[(u, v)] = free_time
            graph.setdefault(u, []).append((v, free_time))
            
all_nodes = set()
for u, v in free_flow_times.keys():
    all_nodes.add(u)
    all_nodes.add(v)
for node in all_nodes:
    graph.setdefault(node, [])

######################################
# 2. OD 페어 읽어와서 필터링 (o != d)
######################################

od_flows = []
with open(trips_file, 'r') as f:
    current_origin = None
    for line in f:
        line = line.strip()
        if not line or line.startswith("~") or line.startswith("<"):
            continue
        if line.lower().startswith("origin"):
            parts = line.split()
            if len(parts) >= 2:
                current_origin = int(parts[1])
        else:
            if current_origin is None:
                continue
            line = line.replace(";", "")
            dest_entries = line.split()
            i = 0
            while i < len(dest_entries):
                try:
                    if dest_entries[i].endswith(':'):
                        dest = int(dest_entries[i].replace(":", ""))
                        flow = float(dest_entries[i+1])
                        i += 2
                    else:
                        dest = int(dest_entries[i])
                        if dest_entries[i+1].startswith(':'):
                            flow = float(dest_entries[i+2])
                            i += 3
                        else:
                            flow = float(dest_entries[i+1])
                            i += 2
                    # origin != destination인 경우만 추가
                    if flow > 1e-6 and current_origin != dest:
                        od_flows.append((current_origin, dest, flow))
                except:
                    i += 1
                    continue

print(f"Loaded {len(all_nodes)} nodes, {len(free_flow_times)} directed links, and {len(od_flows)} OD pairs (o != d).")

######################################
# 3. NetworkX 그래프로 변환 및 경로 계산 함수 정의
######################################
G_nx = nx.DiGraph()
for u, neighbors in graph.items():
    for (v, free_time) in neighbors:
        G_nx.add_edge(u, v, free_flow=free_time)
        
######################################
# 4. 경로 계산 및 후보 경로 생성
######################################

def compute_path_travel_time(path, G):
    return sum(G[u][v]['free_flow'] for u, v in zip(path[:-1], path[1:]))

def generate_candidate_routes(G, origin, destination,
                              max_increase=1.0, max_candidates=50):
    """
    G: NetworkX DiGraph (edge attribute 'cost')
    origin, destination: 노드 ID
    max_increase: 최단 경로 시간 대비 허용 여유 비율
    max_candidates: 최대 후보 경로 수
    반환: ([(path, travel_time), ...], T_star) 또는 ([], None)
    """
    try:
        paths = nx.shortest_simple_paths(G, origin, destination, weight='free_flow')
        best = next(paths)
        T_star = compute_path_travel_time(best, G)

        # 첫 번째, 최단 경로는 무조건 추가
        routes = [(best, T_star)]
        seen_times = {T_star}

        # 이후 경로들 순회
        for path in paths:
            T = compute_path_travel_time(path, G)
            # 허용 범위 초과 시 탐색 중단
            if T > T_star * (1 + max_increase):
                break
            # 같은 통행시간 중복 제거
            if T in seen_times:
                continue
            # 신규 통행시간이면 추가
            routes.append((path, T))
            seen_times.add(T)

            # 후보 개수 한도 도달 시 중단
            if len(routes) >= max_candidates:
                break

        return routes, T_star

    except (nx.NetworkXNoPath, StopIteration):
        return [], None


# 5. 트레이닝 데이터셋 생성
num_samples = 3000
attack_ratio = 0.4  # 원하는 값으로 조정
bound_threshold = 0.2
random.seed(1234)
total_flow = sum(f for (_, _, f) in od_flows)
weights = [f/total_flow for (_, _, f) in od_flows]

training_dataset = []

while len(training_dataset) < num_samples:
    o, d, _ = random.choices(od_flows, weights=weights, k=1)[0]
    candidate_routes, T_star = generate_candidate_routes(G_nx, o, d)
    if T_star is None or not candidate_routes:
        continue

    shortest_path, shortest_time = candidate_routes[0]
    alt_routes = [(path, time) for path, time in candidate_routes if time > shortest_time]
    if not alt_routes:
        continue

    # manipulated travel time: 최단 경로 시간의 (1 + attack_ratio)
    manipulated_time = shortest_time * (1 + attack_ratio)

    training_dataset.append({
        'origin': o,
        'destination': d,
        'shortest_path': shortest_path,
        'shortest_travel_time': shortest_time,
        'alternative_paths': alt_routes,
        'manipulated_travel_time': manipulated_time
    })

# -----------------------------------------
# 1) training_dataset 저장
# -----------------------------------------
with open("training_dataset_Chicago_freeflow.pkl", "wb") as f:
    pickle.dump(training_dataset, f)


Loaded 933 nodes, 2950 directed links, and 93135 OD pairs (o != d).
